In [91]:
import requests
import json
import datetime
import pandas as pd
import pytz

class FastGenie:
    
    def __init__(self, location, start_date):
        street, city, state, zipcode = location.split(',')
        self.street = street.strip()
        self.city = city.strip()
        self.state = state.strip()
        self.zipcode = zipcode.strip()
        self.start_date = start_date
        self.description = "This is very simple tool to generate a time table for nerdy Baha'is during the fast."
        
    def geo_api(self):
        GEO_API = 'https://geocoding.geo.census.gov/geocoder/geographies/address?'
        api_out = requests.get(GEO_API, params={
            'returntype': 'geographies',
            'searchtype': 'address',
            'street': self.street,
            'city': self.city,
            'state': self.state,
            'zip': self.zipcode,
            'benchmark': 'Public_AR_Current',
            'vintage': 'Current_Current',
            'layers': 14,
            'format': 'json',}).json()['result']
        
        lat = api_out['addressMatches'][0]['coordinates']['y']
        lng = api_out['addressMatches'][0]['coordinates']['x']
        
        return lat,lng
    
    def get_sun_schedule(self):
        lat,lng = self.geo_api()
        SUNRISE_API = 'https://api.sunrise-sunset.org/json?'
        series_date = pd.date_range(self.start_date, freq='D', periods=21)
        datelist = series_date.astype(str)
        
        output = list()
        
        # Easier to loop to collect the date, too
        for date in datelist:
            api_out = requests.get(SUNRISE_API, params={'lat': lat,
                                                        'lng': lng,
                                                        'date': date,
                                                        'formatted': 0}).json()['results']
            api_out['date'] = date
            api_out['day_of_week'] = datetime.datetime.strptime(date, '%Y-%m-%d').strftime('%A')
            output.append(api_out)
        return output

    def get_time_table(self):
        good_data = self.get_sun_schedule()
        df = pd.DataFrame(good_data)
        df = df[['day_of_week','date','sunrise','sunset']]
        
        # Set timezone
        tz = pytz.timezone('US/Pacific')
        df['sunrise'] = pd.to_datetime(df['sunrise']).dt.tz_convert('US/Pacific')
        df['sunset'] = pd.to_datetime(df['sunset']).dt.tz_convert('US/Pacific')
        
        # Beautify
        df['sunrise'] = df['sunrise'].apply(lambda x: x.strftime('%I:%M:%S %p'))
        df['sunset'] = df['sunset'].apply(lambda x: x.strftime('%I:%M:%S %p'))
        
        return df

## Enter your address and start date, and run this baby!

In [92]:
YOUR_ADDRESS = '268 Coronet Dr,Los Gatos,CA,95032'
FIRST_DAY_OF_FAST = '2021-03-01'
FastPrinter = FastGenie(YOUR_ADDRESS,FIRST_DAY_OF_FAST)
FastPrinter.get_time_table()

,day_of_week,date,sunrise,sunset
0,Monday,2021-03-01,06:37:10 AM,06:02:33 PM
1,Tuesday,2021-03-02,06:35:47 AM,06:03:31 PM
2,Wednesday,2021-03-03,06:34:23 AM,06:04:29 PM
3,Thursday,2021-03-04,06:32:58 AM,06:05:27 PM
4,Friday,2021-03-05,06:31:33 AM,06:06:25 PM
5,Saturday,2021-03-06,06:30:08 AM,06:07:22 PM
6,Sunday,2021-03-07,06:28:41 AM,06:08:19 PM
7,Monday,2021-03-08,06:27:15 AM,06:09:16 PM
8,Tuesday,2021-03-09,06:25:48 AM,06:10:12 PM
9,Wednesday,2021-03-10,06:24:20 AM,06:11:08 PM
